In [1]:
## load modules

import re
import json
import time
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup as soup
from sklearn.feature_extraction.text import CountVectorizer
from itertools import *
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
## these functions convert API json into friendly structure. 
## taken from http://stackoverflow.com/questions/956867/how-to-get-string-objects-instead-of-unicode-ones-from-json-in-python

def _decode_list(data):
    rv = []
    for item in data:
        if isinstance(item, unicode):
            item = item.encode('utf-8')
        elif isinstance(item, list):
            item = _decode_list(item)
        elif isinstance(item, dict):
            item = _decode_dict(item)
        rv.append(item)
    return rv

def _decode_dict(data):
    rv = {}
    for key, value in data.iteritems():
        if isinstance(key, unicode):
            key = key.encode('utf-8')
        if isinstance(value, unicode):
            value = value.encode('utf-8')
        elif isinstance(value, list):
            value = _decode_list(value)
        elif isinstance(value, dict):
            value = _decode_dict(value)
        rv[key] = value
    return rv

In [3]:
n_topics = 100

In [4]:
## load movie names
titles = pd.read_table('lda-data/movietitles/titlesall.csv', header=None, names=["title"])
## load theta df (docs x topics) from 100-iteration 100-topic lda
theta = pd.read_csv('lda-data/gibbs/theta/theta-df{nt}.csv'.format(nt=n_topics))
phi = pd.read_csv('lda-data/gibbs/phi/phi-df{nt}.csv'.format(nt=n_topics))
## load top 15 words per topic df
topwords = pd.read_csv('lda-data/topwords/top-words-per-topic{nt}.csv'.format(nt=n_topics))
## load all mojo data, 2010-2015 (this has response vars like box office, ratings, etc)
mdata = pd.read_csv('lda-data/mojo/mojoall.csv')
## merge movie names with thetas
title_theta = pd.concat([titles, theta],axis=1)
## merge named thetas with mojo data
merged1 = pd.merge(title_theta, mdata, how="left", left_on="title", right_on="title")

In [5]:
## open topic-details json for editing
with open('lda-data/topic-details.json', 'rb') as fp:
    topic_details = json.load(fp,object_hook=_decode_dict)     

## all json dict indices are strings, even things like 'number of topics in this model'
## t_ix saves 'number of topics in this model'
t_ix = str(theta.shape[1])

In [6]:
## we want to look at correlations between variables - but some variables are categorical
## here we just compile a list of metric variables
corr_feats = []
exclude = (['title','audirate','cldate','critrate','genres',
            'id_imdb','id_rt','mpaa','opdate','revurl','year','budget'])
for col in merged1.columns:
    if col not in exclude:
        corr_feats.append(col)

## now get correlation matrix
corr_matrix = np.corrcoef(merged1.ix[:,corr_feats].as_matrix(),rowvar=0)

In [7]:
corr_cutoff = 0.1
for i in xrange(corr_matrix.shape[0]):
    for j in xrange(corr_matrix.shape[1]):
        if (j>i) and (np.abs(corr_matrix[i,j]) > corr_cutoff) and (re.match('t',corr_feats[i])) and ((corr_feats[j]=="tg")or (corr_feats[j]=="crtiscore") or (corr_feats[j]=="audiscore")):
            
            print 'corr [{i},{j}]: {cor}'.format(i=corr_feats[i],j=corr_feats[j],cor=round(corr_matrix[i,j],3))


corr [t0,tg]: 0.313
corr [t5,audiscore]: -0.139
corr [t8,tg]: 0.236
corr [t13,tg]: 0.24
corr [t15,tg]: -0.101
corr [t16,tg]: 0.13
corr [t16,audiscore]: -0.179
corr [t19,audiscore]: 0.169
corr [t21,audiscore]: 0.109
corr [t22,audiscore]: -0.101
corr [t25,audiscore]: -0.188
corr [t34,audiscore]: 0.11
corr [t43,tg]: 0.205
corr [t51,audiscore]: -0.18
corr [t53,audiscore]: 0.101
corr [t54,audiscore]: -0.138
corr [t56,audiscore]: 0.155
corr [t63,audiscore]: -0.118
corr [t70,audiscore]: -0.107
corr [t75,audiscore]: 0.117
corr [t77,tg]: 0.192
corr [t82,tg]: 0.217
corr [t82,audiscore]: 0.109
corr [t85,tg]: -0.103
corr [t92,tg]: -0.176
corr [t92,audiscore]: 0.259
corr [tg,audiscore]: 0.153
corr [tg_adj,audiscore]: 0.101


In [8]:
## assign values to topic_details

for i in xrange(100):
    tt = str(i) 
        
    if i==0:

        topic_details[t_ix][tt]['crit'] = ['Summer Hollywood blockbuster','Enjoyable, entertaining, harmless fun']
        topic_details[t_ix][tt]['genre'] = ['Possibly Fast and Furious', 'Action/Adventure']

    elif i==1:

        topic_details[t_ix][tt]['crit'] = ['Focus on filmmaking']
        topic_details[t_ix][tt]['genre'] = ['']
        
    elif i==2:

        topic_details[t_ix][tt]['crit'] = ['A commentary on the human condition','Beautiful cinematography']
        topic_details[t_ix][tt]['genre'] = ['Bittersweet realism','Possibly about loss and hardship']

    elif i==3:

        topic_details[t_ix][tt]['crit'] = ['A nuanced exploration of grief and loss','Evokes real feeling']
        topic_details[t_ix][tt]['genre'] = ['Tragedy/Drama', 'Possible father/son story']

    elif i==4:

        topic_details[t_ix][tt]['crit'] = ['Unique and profound']
        topic_details[t_ix][tt]['genre'] = ['']
    
    elif i==5:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Possible biopic']
    
    elif i==6:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==7:

        topic_details[t_ix][tt]['crit'] = ['Breathtaking and spectacular']
        topic_details[t_ix][tt]['genre'] = ['Possible nature movie']

    elif i==8:

        topic_details[t_ix][tt]['crit'] = ['Charming and sweet']
        topic_details[t_ix][tt]['genre'] = ['Animated movie']

    elif i==9:

        topic_details[t_ix][tt]['crit'] = ['Inspirational and emotional']
        topic_details[t_ix][tt]['genre'] = ['Sports movie']
        
    elif i==10:

        topic_details[t_ix][tt]['crit'] = ['Summer blockbuster', 'Visual effects']
        topic_details[t_ix][tt]['genre'] = ['Possible Superhero movie','Smells like Marvel']
        
    elif i==11:

        topic_details[t_ix][tt]['crit'] = ['Smart and engrossing']
        topic_details[t_ix][tt]['genre'] = ['Adaptation of book or play']
        
    elif i==12:

        topic_details[t_ix][tt]['crit'] = ['Acting may feel forced']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==13:

        topic_details[t_ix][tt]['crit'] = ['Fascinating and engaging', 'Personal']
        topic_details[t_ix][tt]['genre'] = ['Documentary']
        
    elif i==14:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Possible dance/musical']
        
    elif i==15:

        topic_details[t_ix][tt]['crit'] = ['Intriguing']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==16:

        topic_details[t_ix][tt]['crit'] = ['Happy ending']
        topic_details[t_ix][tt]['genre'] = ['']
        
    elif i==17:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Possibly British drama']

    elif i==18:

        topic_details[t_ix][tt]['crit'] = ['Magical', 'Beautiful']
        topic_details[t_ix][tt]['genre'] = ['Possibly a fairy tale']

    elif i==19:

        topic_details[t_ix][tt]['crit'] = ['Heartfelt and poignant']
        topic_details[t_ix][tt]['genre'] = ['Coming of age tale']
        
    elif i==20:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Possibly Red series']

    elif i==21:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Drama', 'Indie']

    elif i==22:

        topic_details[t_ix][tt]['crit'] = ['Oscar contender', 'Masterpiece']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==23:

        topic_details[t_ix][tt]['crit'] = ['Interesting storytelling']
        topic_details[t_ix][tt]['genre'] = ['Possibly based on a true story']

    elif i==24:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==25:

        topic_details[t_ix][tt]['crit'] = ['Brutal and bloody']
        topic_details[t_ix][tt]['genre'] = ['Crime thriller']

    elif i==26:

        topic_details[t_ix][tt]['crit'] = ['Enjoyable']
        topic_details[t_ix][tt]['genre'] = ['Possibly involves time machines']

    elif i==27:

        topic_details[t_ix][tt]['crit'] = ['Strong narratives', 'Emotional']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==28:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==29:

        topic_details[t_ix][tt]['crit'] = ['Entertaining']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==30:

        topic_details[t_ix][tt]['crit'] = ['Emotionally satisfying', 'Well-crafted']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==31:

        topic_details[t_ix][tt]['crit'] = ['Strong characters']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==32:

        topic_details[t_ix][tt]['crit'] = ['Ridiculous and dull']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==33:

        topic_details[t_ix][tt]['crit'] = ['Enjoyable']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==34:

        topic_details[t_ix][tt]['crit'] = ['Melodramatic', 'Strong storytelling']
        topic_details[t_ix][tt]['genre'] = ['Drama']
    elif i==35:

        topic_details[t_ix][tt]['crit'] = ['Strong performances']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==36:

        topic_details[t_ix][tt]['crit'] = ['Summer blockbuster']
        topic_details[t_ix][tt]['genre'] = ['Action/adventure']
    elif i==37:

        topic_details[t_ix][tt]['crit'] = ['Gruesome and bloody']
        topic_details[t_ix][tt]['genre'] = ['Horror']

    elif i==38:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Possibly Taken series']
        
    elif i==39:

        topic_details[t_ix][tt]['crit'] = ['Directorial debut']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==40:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Science fiction', 'Space and aliens']
    elif i==41:

        topic_details[t_ix][tt]['crit'] = ['Talented actors']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==42:

        topic_details[t_ix][tt]['crit'] = ['Predictable']
        topic_details[t_ix][tt]['genre'] = ['Possibly Taken series']


    elif i==43:

        topic_details[t_ix][tt]['crit'] = ['Talented ensemble']
        topic_details[t_ix][tt]['genre'] = ['']
        
    elif i==44:

        topic_details[t_ix][tt]['crit'] = ['Silly and ludicrous']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==45:

        topic_details[t_ix][tt]['crit'] = ['Crowd-pleasing']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==46:

        topic_details[t_ix][tt]['crit'] = ['Clever and experimental']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==47:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==48:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Probably a remake']
    elif i==49:

        topic_details[t_ix][tt]['crit'] = ['Gripping']
        topic_details[t_ix][tt]['genre'] = ['Political drama', 'Dealing with complex issues']
    elif i==50:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Musical']
    elif i==51:

        topic_details[t_ix][tt]['crit'] = ['Visually stunning', 'Poetic and striking']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==52:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==53:

        topic_details[t_ix][tt]['crit'] = ['Social commentary']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==54:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Probably directed by Woody Allen']
    elif i==55:

        topic_details[t_ix][tt]['crit'] = ['Social commentary']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==56:

        topic_details[t_ix][tt]['crit'] = ['Pleasant and predictable']
        topic_details[t_ix][tt]['genre'] = ['Romantic comedy']
    elif i==57:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Comedy']
    elif i==58:

        topic_details[t_ix][tt]['crit'] = ['Surprising']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==59:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==60:

        topic_details[t_ix][tt]['crit'] = ['Compelling']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==61:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']
        
    elif i==62:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Probably a foreign film']
    elif i==63:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Indie drama']
    elif i==64:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==65:

        topic_details[t_ix][tt]['crit'] = ['Charming and old-fashioned']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==66:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Corruption of the American dream']
    elif i==67:

        topic_details[t_ix][tt]['crit'] = ['Ambitious']
        topic_details[t_ix][tt]['genre'] = ['Story about hope and tragedy','Possible salvation story']
    elif i==68:

        topic_details[t_ix][tt]['crit'] = ['Themes about open sky']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==69:

        topic_details[t_ix][tt]['crit'] = ["Can't quite pull it together","Flat and formulaic"]
        topic_details[t_ix][tt]['genre'] = []
    elif i==70:

        topic_details[t_ix][tt]['crit'] = ['Artistic in theme and vision']
        topic_details[t_ix][tt]['genre'] = ['A movie about art','Possible biopic']
    elif i==71:

        topic_details[t_ix][tt]['crit'] = ['Lots of plot twists','Plays mind games']
        topic_details[t_ix][tt]['genre'] = ['Thriller', 'Suspense','Possible spy movie']
    elif i==72:

        topic_details[t_ix][tt]['crit'] = ['Reminds us of life to be lived']
        topic_details[t_ix][tt]['genre'] = ['Get up, quit job, follow dreams']
    elif i==73:

        topic_details[t_ix][tt]['crit'] = ['Non-stop entertainment','Shallow thrills']
        topic_details[t_ix][tt]['genre'] = ['Action',"Ridley Scott territory"]
    elif i==74:

        topic_details[t_ix][tt]['crit'] = ['A bit overdone','Grandoise pudding']
        topic_details[t_ix][tt]['genre'] = ['Fantasy','Neverending Story-ish']
    elif i==75:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Epic adventure','Hints of swashbuckle','Possible fantasy/fairytale']
    elif i==76:

        topic_details[t_ix][tt]['crit'] = ['Off-color humor','Borderline tasteless','Cheap gags']
        topic_details[t_ix][tt]['genre'] = ['Raunchy, slapstick comedy','Possibly The Hangover series']
    elif i==77:

        topic_details[t_ix][tt]['crit'] = ['Dull movie']
        topic_details[t_ix][tt]['genre'] = ['Slightly dark']
    elif i==78:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Possible gang/drug themes']
    elif i==79:

        topic_details[t_ix][tt]['crit'] = ['Amazing, strong performance']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==80:

        topic_details[t_ix][tt]['crit'] = ['Good on-screen chemistry']
        topic_details[t_ix][tt]['genre'] = ['Romance flick']
    elif i==81:

        topic_details[t_ix][tt]['crit'] = ['Honest, powerful storytelling']
        topic_details[t_ix][tt]['genre'] = ["Strong female lead", "One woman's journey"]
    elif i==82:

        topic_details[t_ix][tt]['crit'] = ['Compelling look at human nature']
        topic_details[t_ix][tt]['genre'] = ['Possible true story','Moral drama']
    elif i==83:

        topic_details[t_ix][tt]['crit'] = ['Nice, if a bit overdone']
        topic_details[t_ix][tt]['genre'] = ['A story of young love']
    elif i==84:

        topic_details[t_ix][tt]['crit'] = ['Lackluster','(Rare) moments of brightness']
        topic_details[t_ix][tt]['genre'] = ['Either far future or past','Possible sci-fi','Might star Will Smith']
    elif i==85:

        topic_details[t_ix][tt]['crit'] = ['Strong sexual themes','Explores taboos','Possible feminist outrage']
        topic_details[t_ix][tt]['genre'] = ['Steamy','Possibly 50 Shades of Grey']
    elif i==86:

        topic_details[t_ix][tt]['crit'] = ['Hollywood star power','Fun, entertaining romp']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==87:

        topic_details[t_ix][tt]['crit'] = ['A bit campy','Tries but misses the mark']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==88:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Violent thriller','Possible serial killer plot']
    elif i==89:

        topic_details[t_ix][tt]['crit'] = ['Low-budget','Lacks punch']
        topic_details[t_ix][tt]['genre'] = ['Possible Western']
    elif i==90:

        topic_details[t_ix][tt]['crit'] = ['Directorial debut','Strong, creative film']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==91:

        topic_details[t_ix][tt]['crit'] = ['Powerful, absorbing drama']
        topic_details[t_ix][tt]['genre'] = ['War movie','Possibly historical setting']
    elif i==92:

        topic_details[t_ix][tt]['crit'] = ['This is a really bad movie']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==93:

        topic_details[t_ix][tt]['crit'] = ['So-so movie']
        topic_details[t_ix][tt]['genre'] = ['Story of a boy','Possible coming of age story']
    elif i==94:

        topic_details[t_ix][tt]['crit'] = ['Kind of a bad movie','Tired themes']
        topic_details[t_ix][tt]['genre'] = ['']
    elif i==95:

        topic_details[t_ix][tt]['crit'] = ['Thoughtful','Slightly pious']
        topic_details[t_ix][tt]['genre'] = ['Religious theme, probably Christian']
    elif i==96:

        topic_details[t_ix][tt]['crit'] = ['Meh.']
        topic_details[t_ix][tt]['genre'] = ['Franchise sequel','Possible holiday theme']
    elif i==97:

        topic_details[t_ix][tt]['crit'] = ['Kind of a bad movie']
        topic_details[t_ix][tt]['genre'] = ['High school', 'Possibly based on a tv show']
    elif i==98:

        topic_details[t_ix][tt]['crit'] = ['Tackles a meaningful problem','Exposes human nature']
        topic_details[t_ix][tt]['genre'] = ['Outdoorsy','Possible travelogue']
    elif i==99:

        topic_details[t_ix][tt]['crit'] = ['Fast-paced','A little hard to follow']
        topic_details[t_ix][tt]['genre'] = ['Martial arts flick','Chase scenes']
  
    for j in [100,107,108]: # these are the indices in corr_feats for tg, critscore, and audiscore
        
        topic_details[t_ix][tt]['corr'][corr_feats[j]] = round(corr_matrix[i,j],3)

In [9]:
topwords.ix[:,[40,25,79,90]]

,t40,t25,t79,t90
0,women,bad,little,city
1,expect,little,watch,way
2,don,character,time,kind
3,does,barely,way,sound
4,mother,dull,project,room
5,female,good,actually,pleasures
6,think,jason,idea,want
7,know,sense,gives,cinema
8,guys,minutes,getting,plays
9,especially,mess,welcome,right


In [18]:
merged1.ix[merged1.t68> .1, ['title','t68','tg','critscore','audiscore']].head(20)

,title,t68,tg,critscore,audiscore
63,Hubble 3D,0.102113,46370134,86,86
712,Skyfall,0.146154,304360277,92,86
1116,Safe Haven,0.115196,71349120,12,67
1259,Short Term 12,0.204741,1013100,99,92


In [18]:
## save updated topic-details json file
with open('website/web/data/topic-details/topic-details.json', 'wb') as fp:
    json.dump(topic_details, fp)

In [ ]:

'''    elif i==37:

        topic_details[t_ix][tt]['crit'] = ['Might have The Rock in it','Full of cliches']
        topic_details[t_ix][tt]['genre'] = ['Feels like Bromance']

    elif i==43:

        topic_details[t_ix][tt]['crit'] = ['A dumb, hot mess', 'Wasted star power']
        topic_details[t_ix][tt]['genre'] = ['']

    elif i==54:

        topic_details[t_ix][tt]['crit'] = ['']
        topic_details[t_ix][tt]['genre'] = ['Possible conclusion of a series']
    else:
        topic_details[t_ix][tt]['crit'] = [str(i)+'crit']
        topic_details[t_ix][tt]['genre'] = [str(i)+'genre']
  '''    

In [80]:
proceed = False

tops = [37,38,41,49,55,75,74,80,88,90,95]
if proceed:
    for tt_ix in tops:
        tt = str(tt_ix)
        
        '''
        if tt_ix==25:
            
            topic_details[t_ix][tt]['crit'] = ['Silly, dumb fun','Painful scriptwriting','Mindless entertainment']
            topic_details[t_ix][tt]['genre'] = ['Action-comedy']

        if tt_ix==38:
            
            topic_details[t_ix][tt]['crit'] = ['A commentary on the human condition','Beautiful cinematography']
            topic_details[t_ix][tt]['genre'] = ['Bittersweet realism','Possibly about loss and hardship']
            
        if tt_ix==41:
            
            topic_details[t_ix][tt]['crit'] = ['A nuanced exploration of grief and loss','Evokes real feeling']
            topic_details[t_ix][tt]['genre'] = ['Tragedy/Drama', 'Possible father/son story']


        if tt_ix==55:
            
            topic_details[t_ix][tt]['crit'] = ['A nuanced exploration of grief and loss','Evokes real feeling']
            topic_details[t_ix][tt]['genre'] = ['Tragedy/Drama', 'Possible father/son story']

        if tt_ix==75:
            
            topic_details[t_ix][tt]['crit'] = ['Silly, dumb fun','Painful scriptwriting','Mindless entertainment']
            topic_details[t_ix][tt]['genre'] = ['Action-comedy']
        if tt_ix==79:
            
            topic_details[t_ix][tt]['crit'] = ['Summer Hollywood blockbuster','Enjoyable, entertaining, harmless fun']
            topic_details[t_ix][tt]['genre'] = ['Possible superhero movie', 'Action/Adventure']

        if tt_ix==80:
            
            topic_details[t_ix][tt]['crit'] = ['A commentary on the human condition','Beautiful cinematography']
            topic_details[t_ix][tt]['genre'] = ['Bittersweet realism','Possibly about loss and hardship']
            
        if tt_ix==88:
            
            topic_details[t_ix][tt]['crit'] = ['A nuanced exploration of grief and loss','Evokes real feeling']
            topic_details[t_ix][tt]['genre'] = ['Tragedy/Drama', 'Possible father/son story']

        if tt_ix==90:
            
            topic_details[t_ix][tt]['crit'] = ['Silly, dumb fun','Painful scriptwriting','Mindless entertainment']
            topic_details[t_ix][tt]['genre'] = ['Action-comedy']
            
        if tt_ix==95:
            
            topic_details[t_ix][tt]['crit'] = ['Silly, dumb fun','Painful scriptwriting','Mindless entertainment']
            topic_details[t_ix][tt]['genre'] = ['Action-comedy']
        '''
topwords.ix[:,tops]

,t37,t38,t41,t49,t55,t75,t74,t80,t88,t90,t95
0,ve,big,fun,way,little,kids,dead,best,political,horror,original
1,seen,fantasy,stop,hope,make,animated,little,year,moral,house,sequel
2,wish,epic,clever,sense,goes,children,idea,director,powerful,genre,new
3,better,adventure,gets,place,actually,animation,isn,oscar,human,scary,good
4,johnson,cross,good,feel,predictable,disney,sort,films,gripping,filmmakers,time
5,moore,spectacle,entertaining,finds,audience,family,inside,end,power,scares,remake
6,lot,hero,silly,say,doesn,parents,turns,simply,shot,suspense,fun
7,cop,thing,great,right,way,feature,strong,close,truth,zombie,series
8,hard,great,time,country,bit,adults,despite,performances,politics,footage,second
9,right,surprisingly,audience,screen,hangover,charm,director,great,drama,haunted,franchise
